In [1]:
from elasticsearch import helpers, Elasticsearch
import elasticsearch

In [2]:
es = Elasticsearch('http://localhost:9200', max_retries=10, retry_on_timeout=True)

In [3]:
df = spark.read.csv('DNPBA2017.csv', header=True)
df.limit(2).toPandas()

22/05/11 18:24:56 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,ID,NUMERODN,CODINST,ORIGEM,NUMERODV,PREFIXODN,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,...,TPFUNCRESP,TPDOCRESP,DTDECLARAC,ESCMAEAGR1,STDNEPIDEM,STDNNOVA,CODPAISRES,TPROBSON,PARIDADE,KOTELCHUCK
0,1,72390242,ETO1721000001,1,0,30,2786095,170700,1,25,...,2,3,24012017,01,0,1,NA,03,1,3
1,2,72396320,ETO1721000001,1,9,30,NA,291110,3,20,...,2,3,10042017,02,0,1,NA,03,1,2


### indexng

In [4]:
es_index_name = "matb10-sinasc"
df.write.format("org.elasticsearch.spark.sql").option("es.resource", es_index_name).mode('overwrite').save()

### buscando inexata termo

In [5]:
content = { "bool": 
           { "should": [ {"term": {"CODMUNNASC":"291110"}} ] 
           } 
          }

In [6]:
res = es.search(index=es_index_name, query=content, size=2)

In [7]:
res['hits']['hits']

[{'_index': 'matb10-sinasc',
  '_id': 'e60DtYABiDU2XKZNB8zf',
  '_score': 6.518894,
  '_source': {'ID': '2',
   'NUMERODN': '72396320',
   'CODINST': 'ETO1721000001',
   'ORIGEM': '1',
   'NUMERODV': '9',
   'PREFIXODN': '30',
   'CODESTAB': 'NA',
   'CODMUNNASC': '291110',
   'LOCNASC': '3',
   'IDADEMAE': '20',
   'ESTCIVMAE': '2',
   'ESCMAE': '3',
   'CODOCUPMAE': 'NA',
   'QTDFILVIVO': '02',
   'QTDFILMORT': '01',
   'CODMUNRES': '291110',
   'GESTACAO': '5',
   'GRAVIDEZ': '1',
   'PARTO': '1',
   'CONSULTAS': '2',
   'DTNASC': '27032017',
   'HORANASC': '0015',
   'SEXO': '1',
   'APGAR1': 'NA',
   'APGAR5': 'NA',
   'RACACOR': 'NA',
   'PESO': 'NA',
   'IDANOMAL': '2',
   'DTCADASTRO': '06092018',
   'CODANOMAL': 'NA',
   'NUMEROLOTE': '20180030',
   'VERSAOSIST': '3.2.00',
   'DTRECEBIM': '10092018',
   'DIFDATA': '532',
   'DTRECORIGA': 'NA',
   'NATURALMAE': '817',
   'CODMUNNATU': '170700',
   'CODUFNATU': '17',
   'ESCMAE2010': '1',
   'SERIESCMAE': '4',
   'DTNASCMAE': '0

<hr />

### buscando exata termo

In [8]:
content = { "bool": 
           { "must": [ 
               {"match": {"CODESTAB": "2786095"}},
               {"term": {"CODMUNNASC":"170700"}}] 
           } 
          }

In [9]:
res = es.search(index=es_index_name, query=content, size=2)

In [10]:
res['hits']['hits']

[{'_index': 'matb10-sinasc',
  '_id': 'eq0DtYABiDU2XKZNB8zf',
  '_score': 21.229038,
  '_source': {'ID': '1',
   'NUMERODN': '72390242',
   'CODINST': 'ETO1721000001',
   'ORIGEM': '1',
   'NUMERODV': '0',
   'PREFIXODN': '30',
   'CODESTAB': '2786095',
   'CODMUNNASC': '170700',
   'LOCNASC': '1',
   'IDADEMAE': '25',
   'ESTCIVMAE': '5',
   'ESCMAE': '2',
   'CODOCUPMAE': 'NA',
   'QTDFILVIVO': '04',
   'QTDFILMORT': '00',
   'CODMUNRES': '291110',
   'GESTACAO': '5',
   'GRAVIDEZ': '1',
   'PARTO': '1',
   'CONSULTAS': '2',
   'DTNASC': '24012017',
   'HORANASC': '0245',
   'SEXO': '1',
   'APGAR1': '08',
   'APGAR5': '09',
   'RACACOR': '4',
   'PESO': '3150',
   'IDANOMAL': '2',
   'DTCADASTRO': '14022017',
   'CODANOMAL': 'NA',
   'NUMEROLOTE': '20170021',
   'VERSAOSIST': '3.2.00',
   'DTRECEBIM': '10042017',
   'DIFDATA': '076',
   'DTRECORIGA': 'NA',
   'NATURALMAE': '829',
   'CODMUNNATU': '291110',
   'CODUFNATU': '29',
   'ESCMAE2010': '1',
   'SERIESCMAE': '3',
   'DTNASCM

### buscando um intervalo

In [11]:
content = {"range": {
            "IDADEMAE": {
                "gte": 20,
                "lt": 30
            }
        }
    }

In [12]:
res = es.search(index=es_index_name, query=content, size=2)

In [13]:
res['hits']

{'total': {'value': 10000, 'relation': 'gte'},
 'max_score': 1.0,
 'hits': [{'_index': 'matb10-sinasc',
   '_id': 'VK0DtYABiDU2XKZNEPje',
   '_score': 1.0,
   '_source': {'ID': '77367',
    'NUMERODN': '72283388',
    'CODINST': 'MBA2927400003',
    'ORIGEM': '1',
    'NUMERODV': '3',
    'PREFIXODN': '30',
    'CODESTAB': '0003794',
    'CODMUNNASC': '292740',
    'LOCNASC': '1',
    'IDADEMAE': '27',
    'ESTCIVMAE': '1',
    'ESCMAE': '4',
    'CODOCUPMAE': '421105',
    'QTDFILVIVO': '01',
    'QTDFILMORT': 'NA',
    'CODMUNRES': '292740',
    'GESTACAO': '5',
    'GRAVIDEZ': '1',
    'PARTO': '1',
    'CONSULTAS': '4',
    'DTNASC': '12032017',
    'HORANASC': '0340',
    'SEXO': '1',
    'APGAR1': '09',
    'APGAR5': '10',
    'RACACOR': '4',
    'PESO': '3040',
    'IDANOMAL': '2',
    'DTCADASTRO': '14032017',
    'CODANOMAL': 'NA',
    'NUMEROLOTE': '20170023',
    'VERSAOSIST': '3.2.00',
    'DTRECEBIM': '16032017',
    'DIFDATA': '004',
    'DTRECORIGA': 'NA',
    'NATURALMA

### busca lógica

In [14]:
content = {
    "bool": {
            "must": [
                {
                    "term": {
                        "IDADEMAE": 21
                    }
                },
                {
                    "term": {
                        "ESCMAEAGR1": "06"
                    }
                }
            ]
        }
    }

In [15]:
res = es.search(index=es_index_name, query=content, size=2)

In [16]:
res

ObjectApiResponse({'took': 12, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2217, 'relation': 'eq'}, 'max_score': 4.693493, 'hits': [{'_index': 'matb10-sinasc', '_id': 'i60DtYABiDU2XKZNEfpE', '_score': 4.693493, '_source': {'ID': '3649', 'NUMERODN': '74218948', 'CODINST': 'MBA2900700001', 'ORIGEM': '1', 'NUMERODV': '3', 'PREFIXODN': '30', 'CODESTAB': '2487411', 'CODMUNNASC': '290070', 'LOCNASC': '1', 'IDADEMAE': '21', 'ESTCIVMAE': '1', 'ESCMAE': '4', 'CODOCUPMAE': '999992', 'QTDFILVIVO': '01', 'QTDFILMORT': '00', 'CODMUNRES': '290070', 'GESTACAO': '5', 'GRAVIDEZ': '1', 'PARTO': '1', 'CONSULTAS': '3', 'DTNASC': '31052017', 'HORANASC': '1618', 'SEXO': '2', 'APGAR1': '09', 'APGAR5': '10', 'RACACOR': 'NA', 'PESO': '3730', 'IDANOMAL': '2', 'DTCADASTRO': '04072017', 'CODANOMAL': 'NA', 'NUMEROLOTE': '20170017', 'VERSAOSIST': '3.2.00', 'DTRECEBIM': '13072017', 'DIFDATA': '043', 'DTRECORIGA': 'NA', 'NATURALMAE': '829', 'COD

### searching methods

In [17]:
def buscaExata(numerodn, codestab, idademae, escmaeagr1, startId=0): 
    
    global es
    
    content = {
        'bool': {
                'must': [
                    {'match_phrase': {'NUMERODN': '"' + numerodn + '"'}},
                    {'match_phrase': {'CODESTAB': '"' + codestab + '"'}}, 
                    {'match': {'IDADEMAE': idademae}},
                    {'match': {'ESCMAEAGR1': escmaeagr1}}
                ]
            }
        }
    force = True
    while force:
        try:
            res = es.search(index=es_index_name, query=content, size=2)
            force = False
        except:
            pass
    return res['hits']['hits']

In [18]:
def buscaAproximada(numerodn, codestab, idademae, escmaeagr1, startId=0):
    
    global es
    
    content = {
        'bool': {
                'should': [
                    {'match': {'NUMERODN': {'query': numerodn, 'fuzziness':'AUTO', 'operator':'or', 'boost':'2'}}},
                    {'match': {'CODESTAB': {'query': codestab, 'fuzziness':'AUTO', 'operator':'or', 'boost':'2'}}},
                    {'match': {'IDADEMAE': {'query': idademae, 'fuzziness':'AUTO', 'operator':'or', 'boost':'0.5'}}},
                    {'match': {'ESCMAEAGR1': {'query': escmaeagr1, 'fuzziness':'AUTO', 'operator':'or'}}}
                ]
            }
        }
    force = True
    while force:
        try:
            res = es.search(index=es_index_name, query=content, size=2)
            force = False
        except:
            pass
    return res['hits']['hits']

In [19]:
buscaExata("72390242", "2786095", "25", "01")

[{'_index': 'matb10-sinasc',
  '_id': 'eq0DtYABiDU2XKZNB8zf',
  '_score': 29.667755,
  '_source': {'ID': '1',
   'NUMERODN': '72390242',
   'CODINST': 'ETO1721000001',
   'ORIGEM': '1',
   'NUMERODV': '0',
   'PREFIXODN': '30',
   'CODESTAB': '2786095',
   'CODMUNNASC': '170700',
   'LOCNASC': '1',
   'IDADEMAE': '25',
   'ESTCIVMAE': '5',
   'ESCMAE': '2',
   'CODOCUPMAE': 'NA',
   'QTDFILVIVO': '04',
   'QTDFILMORT': '00',
   'CODMUNRES': '291110',
   'GESTACAO': '5',
   'GRAVIDEZ': '1',
   'PARTO': '1',
   'CONSULTAS': '2',
   'DTNASC': '24012017',
   'HORANASC': '0245',
   'SEXO': '1',
   'APGAR1': '08',
   'APGAR5': '09',
   'RACACOR': '4',
   'PESO': '3150',
   'IDANOMAL': '2',
   'DTCADASTRO': '14022017',
   'CODANOMAL': 'NA',
   'NUMEROLOTE': '20170021',
   'VERSAOSIST': '3.2.00',
   'DTRECEBIM': '10042017',
   'DIFDATA': '076',
   'DTRECORIGA': 'NA',
   'NATURALMAE': '829',
   'CODMUNNATU': '291110',
   'CODUFNATU': '29',
   'ESCMAE2010': '1',
   'SERIESCMAE': '3',
   'DTNASCM

In [20]:
# um campo modificado (note que o retorno é vazio)
buscaExata("72390242", "2786095", "33", "01")

[]

In [21]:
buscaAproximada("72390242", "2786095", "25", "01")

[{'_index': 'matb10-sinasc',
  '_id': 'eq0DtYABiDU2XKZNB8zf',
  '_score': 49.809166,
  '_source': {'ID': '1',
   'NUMERODN': '72390242',
   'CODINST': 'ETO1721000001',
   'ORIGEM': '1',
   'NUMERODV': '0',
   'PREFIXODN': '30',
   'CODESTAB': '2786095',
   'CODMUNNASC': '170700',
   'LOCNASC': '1',
   'IDADEMAE': '25',
   'ESTCIVMAE': '5',
   'ESCMAE': '2',
   'CODOCUPMAE': 'NA',
   'QTDFILVIVO': '04',
   'QTDFILMORT': '00',
   'CODMUNRES': '291110',
   'GESTACAO': '5',
   'GRAVIDEZ': '1',
   'PARTO': '1',
   'CONSULTAS': '2',
   'DTNASC': '24012017',
   'HORANASC': '0245',
   'SEXO': '1',
   'APGAR1': '08',
   'APGAR5': '09',
   'RACACOR': '4',
   'PESO': '3150',
   'IDANOMAL': '2',
   'DTCADASTRO': '14022017',
   'CODANOMAL': 'NA',
   'NUMEROLOTE': '20170021',
   'VERSAOSIST': '3.2.00',
   'DTRECEBIM': '10042017',
   'DIFDATA': '076',
   'DTRECORIGA': 'NA',
   'NATURALMAE': '829',
   'CODMUNNATU': '291110',
   'CODUFNATU': '29',
   'ESCMAE2010': '1',
   'SERIESCMAE': '3',
   'DTNASCM

In [22]:
# # um campo modificado (mais de um candidato é retornado)
buscaAproximada("72390242", "2786095", "33", "01")

[{'_index': 'matb10-sinasc',
  '_id': 'eq0DtYABiDU2XKZNB8zf',
  '_score': 48.27685,
  '_source': {'ID': '1',
   'NUMERODN': '72390242',
   'CODINST': 'ETO1721000001',
   'ORIGEM': '1',
   'NUMERODV': '0',
   'PREFIXODN': '30',
   'CODESTAB': '2786095',
   'CODMUNNASC': '170700',
   'LOCNASC': '1',
   'IDADEMAE': '25',
   'ESTCIVMAE': '5',
   'ESCMAE': '2',
   'CODOCUPMAE': 'NA',
   'QTDFILVIVO': '04',
   'QTDFILMORT': '00',
   'CODMUNRES': '291110',
   'GESTACAO': '5',
   'GRAVIDEZ': '1',
   'PARTO': '1',
   'CONSULTAS': '2',
   'DTNASC': '24012017',
   'HORANASC': '0245',
   'SEXO': '1',
   'APGAR1': '08',
   'APGAR5': '09',
   'RACACOR': '4',
   'PESO': '3150',
   'IDANOMAL': '2',
   'DTCADASTRO': '14022017',
   'CODANOMAL': 'NA',
   'NUMEROLOTE': '20170021',
   'VERSAOSIST': '3.2.00',
   'DTRECEBIM': '10042017',
   'DIFDATA': '076',
   'DTRECORIGA': 'NA',
   'NATURALMAE': '829',
   'CODMUNNATU': '291110',
   'CODUFNATU': '29',
   'ESCMAE2010': '1',
   'SERIESCMAE': '3',
   'DTNASCMA

In [23]:
print(len(buscaAproximada("72390242", "2786095", "33", "01")))

2
